# Scrapping a URL



In [87]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# Configuración de Selenium con ChromeDriver Manager
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Ejecutar en modo sin cabeza
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


In [92]:
import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv('bbdd/productos_link.csv', sep=';')  # Ajusta el delimitador si es necesario

df['Link'] = df['Link'].apply(lambda x: 'https://www.solotodo.cl' + x if x.startswith('/product') else x)

# Verificar los primeros resultados
print(df['Link'].head())


# Convertir la columna 'Link' de product_links a una lista
product_links = df['Link'].tolist()

# Verificar el tipo y contenido de product_links2
print("Tipo de product_links:", type(product_links))
print("Contenido de product_links:", product_links)



0    https://www.solotodo.cl/products/83233-lg-k51s...
1    https://www.solotodo.cl/products/215742-bmobil...
2    https://www.solotodo.cl/products/183579-motoro...
3    https://www.solotodo.cl/products/235468-motoro...
4    https://www.solotodo.cl/products/248505-motoro...
Name: Link, dtype: object
Tipo de product_links: <class 'list'>
Contenido de product_links: ['https://www.solotodo.cl/products/83233-lg-k51s-k510hm-azul', 'https://www.solotodo.cl/products/215742-bmobile-c41-dark-gray', 'https://www.solotodo.cl/products/183579-motorola-moto-e13-64-gb-2-gb-aurora-green', 'https://www.solotodo.cl/products/235468-motorola-moto-g04-128-gb-4-gb-concord-black', 'https://www.solotodo.cl/products/248505-motorola-moto-e14-64-gb-2-gb-graphite-gray', 'https://www.solotodo.cl/products/245327-motorola-moto-g04s-128-gb-4-gb-concord-black-pb380028cl', 'https://www.solotodo.cl/products/227597-zte-blade-a54-128-gb-4-gb-black', 'https://www.solotodo.cl/products/155478-motorola-moto-e22i-64-gb-2-gb-gr

In [93]:
# Lista de tiendas y marcas de interés
relevant_entities = [
    "hites", "abcdin", "lider", "paris", "ripley", "falabella", "walmart", "jumbo",
    "samsung", "motorola", "xiaomi", "oppo", "honor", "huawei", "apple", "realme", "oneplus", "nokia", "sony", "lg"
]

In [94]:
# Lista para almacenar los datos extraídos
data = []

# Recorrer cada enlace de producto
for link in product_links:
    driver.get(link)
    time.sleep(2)  # Espera para que la página de cada producto cargue
    
    # Extraer la información HTML de la página con BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Extraer el nombre del equipo
    equipo = soup.find('h2').get_text(strip=True) if soup.find('h2') else "Equipo no disponible"
    
    # Inicializar diccionario para almacenar especificaciones del producto
    product_data = {
        'URL': link,
        'Equipo': equipo,
        'Pantalla': None,
        'Memoria_interna': None,
        'RAM': None,
        'Cámara_principal': None,
        'Cámara_frontal': None,
        'Batería': None,
        'Sistema_operativo': None,
        'Conectividad_celular': None,
        'Procesador': None,
        'CPU': None,
        'GPU': None,
        'Peso': None,
        'Tamaño': None,
        'Soporte_eSIM': None,
        'Potencia_de_carga': None,
        'Almacenamiento_externo': None,
        'Conector_de_audífonos': None,
        'Wi-Fi': None,
        'GPS': None,
        'NFC': None,
        'Bluetooth': None,
        'Lector_de_huellas_digitales': None,
        'Tiendas_y_Precios': None
    }
    # Función auxiliar para extraer valores de cada sección
    def extract_spec(section, product_data):
        for dt, dd in zip(section.find_all("dt"), section.find_all("dd")):
            key = dt.get_text(strip=True).replace(" ", "_").replace(".", "").replace("(", "").replace(")", "")
            value = dd.get_text(strip=True)
            if key in product_data:
                product_data[key] = value

    # Extraer cada sección de características
    sections = soup.find_all("div", class_="col-12")
    for section in sections:
        header = section.find("h3").get_text(strip=True) if section.find("h3") else ""
        if "Características" in header:
            extract_spec(section, product_data)
        elif "Características avanzadas" in header:
            extract_spec(section, product_data)
        elif "Accesorios" in header:
            extract_spec(section, product_data)

    # Extraer tiendas y precios en una sola celda, filtrando por entidades relevantes
    tiendas_precios = []
    for store_section in soup.find_all('a', href=True):
        store_link = store_section['href']
        
        # Validar si el enlace contiene alguna tienda o marca relevante y no está vacío
        if any(entity in store_link.lower() for entity in relevant_entities):
            # Extraer nombre de la tienda y precio solo si existen y no están vacíos
            tienda = store_section.find('p', class_='storeName').get_text(strip=True) if store_section.find('p', class_='storeName') else "Desconocida"
            precio = store_section.find('h2', class_='MuiTypography-h2').get_text(strip=True) if store_section.find('h2', class_='MuiTypography-h2') else "Precio no disponible"
            if tienda != "Desconocida" and precio != "Precio no disponible":
                tiendas_precios.append(f"{tienda}: {precio} ({store_link})")

    # Combinar tiendas y precios en una sola celda
    product_data['Tiendas_y_Precios'] = "; ".join(tiendas_precios) if tiendas_precios else "Sin datos de precios"

    # Agregar el diccionario del producto a la lista de datos
    data.append(product_data)

# Guardar los datos en un archivo CSV
df = pd.DataFrame(data)



In [1]:
print(df)

NameError: name 'df' is not defined

In [100]:
df.to_csv("bbdd/productos_celulares.csv", sep=";", index=False, encoding="utf-8")
